In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
aapl = yf.Ticker("AAPL")

from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [43]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.width',300, 'display.max_columns',500)
%matplotlib inline

In [44]:
#import data 
Option_data=pd.read_csv('NBOption.csv')
Option_data

,DataType,Ask,Bid,LastPrice,StockPrice,Strike,Volatility,Volume,time_to_maturity,ImpliedVolatility
0,put,0.10,0.0,0.11,79.40,30.0,0.284654,22,0.128767,1.242191
1,put,0.25,0.0,0.10,71.86,35.0,0.400073,61,0.090411,1.173832
2,put,1.80,0.0,0.40,79.40,40.0,0.284654,5,0.128767,1.442386
3,put,0.20,0.0,0.09,79.54,45.0,0.286563,4,0.123288,0.763674
4,put,0.15,0.0,0.10,72.10,50.0,0.396490,1,0.095890,0.578129
...,...,...,...,...,...,...,...,...,...,...
14248,call,0.10,0.0,0.05,19.26,25.0,0.198330,6,0.602740,0.233406
14249,call,0.30,0.2,0.20,21.20,30.0,0.247322,10,1.413699,0.435553
14250,call,0.15,0.0,0.05,19.74,35.0,0.200913,1,0.936986,0.461919
14251,call,0.35,0.0,0.35,1.86,2.5,0.502372,10,0.128767,1.765626


In [45]:
#remove unnecessary columns
Option_data_new=Option_data.drop('DataType', axis=1).copy()
Option_data_new.head()

,Ask,Bid,LastPrice,StockPrice,Strike,Volatility,Volume,time_to_maturity,ImpliedVolatility
0,0.10,0.0,0.11,79.40,30.0,0.284654,22,0.128767,1.242191
1,0.25,0.0,0.10,71.86,35.0,0.400073,61,0.090411,1.173832
2,1.80,0.0,0.40,79.40,40.0,0.284654,5,0.128767,1.442386
3,0.20,0.0,0.09,79.54,45.0,0.286563,4,0.123288,0.763674
4,0.15,0.0,0.10,72.10,50.0,0.396490,1,0.095890,0.578129


In [46]:
# Check the basic statistics of data
Option_data_new.describe().T

,count,mean,std,min,25%,50%,75%,max
Ask,14253.0,6.104787,13.483170,0.000000,0.300000,1.450000,5.400000,163.700000
Bid,14253.0,5.509981,12.817976,0.000000,0.110000,1.080000,4.750000,160.500000
LastPrice,14253.0,5.568519,12.001531,0.010000,0.300000,1.350000,5.100000,199.040000
StockPrice,14253.0,66.751016,49.793904,0.380000,25.690000,61.360000,87.120000,240.440000
Strike,14253.0,62.330004,48.064804,0.500000,25.000000,55.000000,82.500000,330.000000
Volatility,14253.0,0.347167,0.299735,0.027001,0.180645,0.270757,0.399859,2.397045
Volume,14253.0,46.547814,255.312600,0.000000,2.000000,7.000000,20.000000,10893.000000
time_to_maturity,14253.0,0.406658,0.459389,0.013699,0.084932,0.235616,0.583562,2.334247
ImpliedVolatility,14253.0,0.448240,0.479725,0.000000,0.228279,0.344733,0.519536,13.437502


In [47]:
#Normalization using MinMaxScaler
normalization_bit=1

if (normalization_bit == 1):
  Option_data_normalized = MinMaxScaler().fit_transform(Option_data_new)
else: 
  Option_data_normalized = StandardScaler().fit_transform(Option_data_new)

Option_data_normalized

array([[6.10873549e-04, 0.00000000e+00, 5.02436819e-04, ...,
        2.01964564e-03, 4.95867768e-02, 9.24421314e-02],
       [1.52718387e-03, 0.00000000e+00, 4.52193137e-04, ...,
        5.59992656e-03, 3.30578513e-02, 8.73549482e-02],
       [1.09957239e-02, 0.00000000e+00, 1.95950359e-03, ...,
        4.59010374e-04, 4.95867768e-02, 1.07340311e-01],
       ...,
       [9.16310324e-04, 0.00000000e+00, 2.00974727e-04, ...,
        9.18020747e-05, 3.97874852e-01, 3.43753963e-02],
       [2.13805742e-03, 0.00000000e+00, 1.70828518e-03, ...,
        9.18020747e-04, 4.95867768e-02, 1.31395420e-01],
       [4.58155162e-03, 0.00000000e+00, 1.70828518e-03, ...,
        3.67208299e-04, 2.64462810e-01, 7.84887145e-02]])

In [48]:
#split the data
from sklearn.model_selection import train_test_split

data=pd.DataFrame(data=Option_data_normalized, columns=Option_data_new.columns)
(n,p)=data.shape

X=data.iloc[:, 0:p-1]


# Label
impliedVolatility = data["ImpliedVolatility"]
volatility = data["Volatility"]

# Split the data set into training data(80%) and testing data(20%)

train_data, test_data, train_data_impliedVolatility, test_data_impliedVolatility = train_test_split(X, impliedVolatility, 
                                                test_size=0.2, random_state=42)
print("\nChecking training and test data info!\n")
print(train_data.shape, test_data.shape)
print(train_data_impliedVolatility.shape, test_data_impliedVolatility.shape)


Checking training and test data info!

(11402, 8) (2851, 8)
(11402,) (2851,)


In [92]:
#Random forests using 30 decision trees

from sklearn.ensemble import RandomForestRegressor

# initailize a random forests regressor
params = {'n_estimators': 30, 
          'max_depth':8,
          'min_samples_split': 2}

RF = RandomForestRegressor(**params)

# fit: do training
RF.fit(train_data,train_data_impliedVolatility)

# predict: predict implied volatilities for test data 
predictedImpliedVotatility= RF.predict(test_data)
Error_RS=np.abs(predictedImpliedVotatility- test_data_impliedVolatility) 

In [93]:
# Checking regression score
from sklearn.metrics import r2_score
print ('The R2 score is: ',(r2_score(test_data_impliedVolatility,predictedImpliedVotatility)))
rf_r2 = r2_score(test_data_impliedVolatility,predictedImpliedVotatility)

The R2 score is:  0.379436964044217


In [94]:
#bad guys
Error_RS = Error_RS.sort_values(ascending=False)
print ('The bad guys are')
(Error_RS[0:round(len(Error_RS)*0.1)].head(10))

The bad guys are


6072     0.478909
6073     0.438460
4546     0.344122
4739     0.343208
11539    0.280303
4767     0.277144
6076     0.239500
4738     0.191494
11465    0.149364
5657     0.136961
Name: ImpliedVolatility, dtype: float64

In [95]:
#KNN
from sklearn.neighbors import KNeighborsRegressor # initailize a kNN regressor

kNN = KNeighborsRegressor(n_neighbors=10, weights='distance', metric='manhattan') # 
kNN.fit(train_data,train_data_impliedVolatility)
# predict: predict implied volatilities for test data
predictedImpliedVotatility = kNN.predict(test_data)
r2score=RF.score(test_data, test_data_impliedVolatility)

In [96]:
Error_KNN=np.abs(predictedImpliedVotatility- test_data_impliedVolatility) 
print('The mean error is ' + str(np.mean(Error_KNN)))

MSE_KNN=np.sum(Error_KNN**2)/len(Error_KNN) 
print('The MSE is {:20.16f}'.format(MSE_KNN))
print('The maximum error is {:20.16f}'.format(max(Error_KNN))) 
print('The minimum error is {:20.16f}'.format(min(Error_KNN)))

The mean error is 0.009113613310551319
The MSE is   0.0004406064090945
The maximum error is   0.3547600341645887
The minimum error is   0.0000000000000000


In [97]:
# Checking regression score
from sklearn.metrics import r2_score
print ('The R2 score is: ',(r2_score(test_data_impliedVolatility,predictedImpliedVotatility)))
knn_r2 = r2_score(test_data_impliedVolatility,predictedImpliedVotatility)

The R2 score is:  0.5963422759615365


In [98]:
#print (Error_KNN)
Error_KNN = Error_KNN.sort_values(ascending=False)
print ('The bad guys are')
(Error_KNN[0:round(len(Error_KNN)*0.1)].head(10))

The bad guys are


4546     0.354760
4767     0.267710
6071     0.224326
13171    0.216558
6072     0.208496
6073     0.202161
11465    0.184223
13195    0.163274
6076     0.149432
5657     0.147282
Name: ImpliedVolatility, dtype: float64

In [99]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(n_estimators = 2000, max_depth = 8, learning_rate = 0.01, loss = 'ls')
gb.fit(train_data,train_data_impliedVolatility)
predictedImpliedVotatility= gb.predict(test_data)

Error_GB=np.abs(predictedImpliedVotatility- test_data_impliedVolatility) 
print('The mean error is ' + str(np.mean(Error_GB)))

MSE_GB=np.sum(Error_GB**2)/len(Error_GB) 
print('The MSE is {:20.16f}'.format(MSE_GB))
print('The maximum error is {:20.16f}'.format(max(Error_GB))) 
print('The minimum error is {:20.16f}'.format(min(Error_GB)))

The mean error is 0.008599328888317284
The MSE is   0.0005201185017149
The maximum error is   0.4604205251849869
The minimum error is   0.0000005668391617


In [100]:
# Checking regression score
from sklearn.metrics import r2_score
print ('The R2 score is: ',(r2_score(test_data_impliedVolatility,predictedImpliedVotatility)))
gb_r2 = r2_score(test_data_impliedVolatility,predictedImpliedVotatility)

The R2 score is:  0.5234979648526101


In [101]:
Error_GB = Error_GB.sort_values(ascending=False)
print ('The bad guys are')
(Error_GB[0:round(len(Error_GB)*0.1)].head(10))

The bad guys are


6072     0.460421
6073     0.397395
11539    0.351319
4546     0.348065
13195    0.231220
4739     0.223653
6076     0.194456
4738     0.177349
6070     0.176554
4742     0.161392
Name: ImpliedVolatility, dtype: float64

In [105]:
#DNN

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

train_data, test_data, train_data_impliedVolatility, test_data_impliedVolatility = train_test_split(X, impliedVolatility, 
                                                test_size=0.2, random_state=42)

reg = MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu" ,random_state=1, max_iter=2000).fit(train_data, train_data_impliedVolatility)

predictedImpliedVotatility = reg.predict(test_data)

In [106]:
#R2 score
print ('The R2 score is: ',(r2_score(test_data_impliedVolatility,predictedImpliedVotatility)))
dnn_r2 = r2_score(test_data_impliedVolatility,predictedImpliedVotatility)

The R2 score is:  0.3581678787814885


In [107]:
#Bad guys
Error_DNN=np.abs(predictedImpliedVotatility- test_data_impliedVolatility) 
#print('The mean error is ' + str(np.mean(Error_GB)))
Error_DNN = Error_DNN.sort_values(ascending=False)
print ('The bad guys are')
(Error_DNN[0:round(len(Error_GB)*0.1)].head(10))

The bad guys are


4546     0.357495
10143    0.303496
4767     0.295878
6072     0.238160
6073     0.236070
11465    0.188359
6071     0.185535
6076     0.173434
10550    0.170093
592      0.164869
Name: ImpliedVolatility, dtype: float64

In [137]:
scores = [knn_r2, dnn_r2, gb_r2, rf_r2]
learning_method = ['KNN', 'DNN', 'Gradient Boosting', 'Random Forest']


scores_df = pd.DataFrame(data = learning_method, columns = ['Learning Method'])
scores_df['R2 Score'] = scores
scores_df = scores_df.sort_values(by = ['R2 Score'],ascending = False)
scores_df

,Learning Method,R2 Score
0,KNN,0.596342
2,Gradient Boosting,0.523498
3,Random Forest,0.379437
1,DNN,0.358168


In [300]:
#SVM
option_data = Option_data_normalized
option_label = data["ImpliedVolatility"]
k = 10
test_percent=0.2
option_training_data,option_test_data,option_training_data_label, option_test_data_label = train_test_split(option_data,option_label, test_size=test_percent, random_state=42)

In [301]:
from timeit import default_timer as timer
from sklearn.metrics import mean_squared_error


test_percent=0.4
option_training_data,option_test_data,option_training_data_label, option_test_data_label=train_test_split(option_data,option_label, test_size=test_percent, random_state=42)


kernel_list=['linear','rbf','poly','sigmoid']
print('SVM results\n')

result=pd.DataFrame(columns=['score','running time','mse',])
for kernel in kernel_list:
    clf=svm.SVR(kernel=kernel,tol=0.0001,gamma='auto')
    start=timer()
    clf.fit(option_training_data,option_training_data_label)
    predicted_label=clf.predict(option_test_data)
    end=timer()
    mse=mean_squared_error(option_test_data_label, predicted_label)
    running_time=end-start
    score = clf.score(option_test_data,option_test_data_label)
    result.loc[kernel]={'running time':running_time,'mse':mse,'score':score}
    
result

SVM results



,score,running time,mse
linear,-5.990352,0.009379,0.007841
rbf,-5.006340,0.014263,0.006738
poly,-4.010061,0.043014,0.005620
sigmoid,-5.295825,0.016309,0.007062


In [394]:
#Download data from Yahoo finance
aapl = yf.Ticker("AAPL")
#aapl.options
opt = aapl.option_chain('2021-04-30')
option_data = opt.calls 
option_data = pd.DataFrame(data = option_data)
option_data = option_data.iloc[:,2::]
#option_data = option_data - option_data.iloc[:,-3::]
option_data = option_data.drop(columns = ['inTheMoney', 'contractSize','currency'])
print ('The AAPL option data')
option_data.head(5)

The AAPL option data


,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility
0,70.0,51.70,55.45,55.60,0.000000,0.000000,1,3,0.00001
1,75.0,45.20,50.40,50.50,0.000000,0.000000,20,20,0.00001
2,80.0,45.05,45.45,45.50,2.200001,5.134191,5,101,0.00001
3,85.0,38.25,40.40,40.55,0.000000,0.000000,1,190,0.00001
4,90.0,36.20,35.50,35.65,2.549999,7.578006,7,22,0.00001


In [397]:
normalization_bit= 0

if (normalization_bit == 1):
    Option_data_normalized = MinMaxScaler().fit_transform(option_data)
else: 
    Option_data_normalized = StandardScaler().fit_transform(option_data)

df = pd.DataFrame(data = Option_data_normalized, columns = option_data.columns)
print ('The Normalized AAPL Data')
df.head(5)

The Normalized AAPL Data


,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility
0,-2.227852,3.119527,3.192258,3.189334,-1.155716,-1.249889,-0.509679,-0.604412,-2.209048
1,-2.017758,2.620948,2.826874,2.821177,-1.155716,-1.249889,-0.499233,-0.599133,-2.209048
2,-1.807664,2.609442,2.468725,2.460240,1.160014,-0.989668,-0.507480,-0.573980,-2.209048
3,-1.597570,2.087851,2.103340,2.102911,-1.155716,-1.249889,-0.509679,-0.546343,-2.209048
4,-1.387477,1.930607,1.748809,1.749192,1.528423,-0.865806,-0.506380,-0.598512,-2.209048


In [392]:
option_label = option_data['impliedVolatility']
option_data = Option_data_normalized
#option_data

In [393]:
k = 10
test_percent=0.4
option_training_data,option_test_data,option_training_data_label, option_test_data_label=train_test_split(option_data,option_label, test_size=test_percent, random_state=42)


kernel_list=['linear','rbf','poly','sigmoid']
print('SVM results\n')

result=pd.DataFrame(columns=['score','running time','mse',])
for kernel in kernel_list:
    clf=svm.SVR(kernel=kernel,tol=0.0001,gamma='auto')
    start=timer()
    clf.fit(option_training_data,option_training_data_label)
    predicted_label=clf.predict(option_test_data)
    end=timer()
    mse=mean_squared_error(option_test_data_label, predicted_label)
    running_time=end-start
    score = clf.score(option_test_data,option_test_data_label)
    result.loc[kernel]={'running time':running_time,'mse':mse,'score':score}
    
result

SVM results



,score,running time,mse
linear,0.554063,0.001246,0.002639
rbf,0.295212,0.000592,0.004171
poly,0.168841,0.000533,0.004919
sigmoid,-38.245979,0.001325,0.232270
